# Análisis de Escritos según Ámbito 1: Aspectos de Estructura Global

El prompt, formulado usando CHATGPT, permite evaluar un texto de manera automática según los tres criterios de la rúbrica académica:  

1. **Estructura global: Introducción**  
   Evalúa si el texto presenta una introducción clara, bien contextualizada o simplemente mencionada.  

2. **Estructura global: Conclusión o cierre**  
   Revisa si el texto cierra correctamente con una conclusión, una proyección de la tesis o solo un comentario.  

3. **Progresión textual**  
   Analiza el uso de párrafos y si cada uno aporta una idea principal distinta.  

El resultado de la evaluación se entrega en **formato JSON** para facilitar su interpretación y posterior uso en sistemas automáticos o reportes.  

In [5]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_con_reactivo.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()
reactivos = df["reactivo"].fillna("").tolist()

prompt_estructura = """
Eres un evaluador EXTREMADAMENTE ESTRICTO de escritura académica. 
Tu tarea es evaluar y detectar deficiencias en tres dimensiones: **introducción**, **conclusión y cierre**, y **progresión textual**, verificando además que el texto responda adecuadamente al REACTIVO. 
No celebres logros parciales: tu foco es identificar falencias con precisión.

---

## 1. INTRODUCCIÓN

Una introducción completa debe cumplir con cuatro elementos esenciales:  
1. **Qué**: definir claramente el tema central en una o dos oraciones precisas.  
2. **Quiénes**: mencionar los actores, personas o grupos relevantes por nombre.  
3. **Dónde**: señalar el lugar o contexto espacial específico.  
4. **Cuándo**: indicar el periodo temporal, fecha o momento histórico.  

A partir de la presencia y calidad de estos elementos, se asignan los niveles:  
- **Nivel 4**: todos los elementos están presentes y son específicos.  
- **Nivel 3**: tres elementos presentes o dos o tres elementos vagos o generales.  
- **Nivel 2**: solo uno o dos elementos identificables.  
- **Nivel 1**: ningún elemento claro o el texto comienza abruptamente.  

*PRECAUCIÓN*: mencionar el tema no significa tener una introducción completa.

---

## 2. CONCLUSIÓN Y CIERRE

Una conclusión eficaz debe cumplir dos funciones principales:  
1. **Síntesis**: reformular la idea principal en palabras nuevas.  
2. **Proyección**: indicar implicaciones, consecuencias, reflexiones o contextos futuros relacionados con el tema.  

Según cómo cumpla estas funciones, se asignan los niveles:  
- **Nivel 4**: síntesis clara y proyección explícita.  
- **Nivel 3**: solo síntesis o solo proyección bien fundamentada.  
- **Nivel 2**: hay un intento de cierre pero sin estructura clara.  
- **Nivel 1**: no se identifica conclusión ni cierre.

*PRECAUCIÓN*: una oración final simple no equivale a una conclusión de nivel 4.

---

## 3. PROGRESIÓN TEXTUAL

Un texto con buena progresión debe cumplir con:  
1. Cada párrafo desarrolla un subtema distinto.  
2. No se repiten ideas entre párrafos.  
3. Hay conectores lógicos que faciliten la lectura y la coherencia entre párrafos.  

Los niveles se asignan en función de estos criterios:  
- **Nivel 4**: todos los párrafos aportan información nueva y diferente.  
- **Nivel 3**: un párrafo repite parcialmente información de otro.  
- **Nivel 2**: dos o más párrafos son redundantes o no hay separación clara.  
- **Nivel 1**: texto sin estructura de párrafos o totalmente repetitivo.

*PRECAUCIÓN*: si dos párrafos dicen básicamente lo mismo con palabras distintas, el nivel máximo es 3.

---

## INSTRUCCIONES CRÍTICAS

1. Evalúa cada dimensión **en función de los elementos que deberían estar presentes** y su correspondencia con el REACTIVO.  
2. Si dudas entre dos niveles, elige el menor.  
3. No sobreestimes textos mediocres; la mayoría se sitúa entre niveles 2 y 3.  
4. Justifica cada nivel señalando las deficiencias específicas.  
5. No inventes información, pero puedes sugerir correcciones breves para que el texto cumpla con los criterios.

---

## FORMATO DE SALIDA

Devuelve **solo un JSON válido**, sin markdown ni comentarios adicionales:

{{
  "introduccion": {{
    "nivel": X,
    "justificacion": "Indica qué elementos faltan, están vagos o no responden al reactivo"
  }},
  "conclusion_cierre": {{
    "nivel": X,
    "justificacion": "Indica si falta síntesis, proyección, ambos o si no responde al reactivo"
  }},
  "progresion_textual": {{
    "nivel": X,
    "justificacion": "Señala qué párrafos repiten información, si falta estructura o si el texto no responde al reactivo"
  }}
}}

---

## REACTIVO

{reactive}

---

## TEXTO A EVALUAR

{essay}

---

## RECORDATORIO FINAL

- Nivel 4 requiere cumplimiento completo, explícito y pertinente de todos los criterios.  
- Sé específico en tus justificaciones y apunta siempre a identificar deficiencias.
"""

resultados = []

for i, (texto, reactivo) in enumerate(zip(textos, reactivos), start=1):    
    prompt_estructura_a1 = prompt_estructura.format(essay=texto, reactive=reactivo)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza la evaluación del texto de acuerdo con los criterios estrictos proporcionados'},
        {"role": "user", "content": prompt_estructura_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_estructura.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_estructura.json


In [6]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_con_reactivo.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()
reactivos = df["reactivo"].fillna("").tolist()

prompt_introduccion = """
Eres un evaluador y corrector EXTREMADAMENTE ESTRICTO de escritura académica. 
Tu tarea es evaluar la **INTRODUCCIÓN** de un texto para que cumpla con los siguientes criterios académicos exigidos y responda adecuadamente al REACTIVO planteado.

--- 

## CRITERIO DE EVALUACIÓN DE INTRODUCCIÓN

La calidad de la introducción se evaluará exclusivamente en función de la presencia de los siguientes cuatro elementos:

1. **Qué**: tema central claramente definido en 1-2 oraciones específicas.
2. **Quiénes**: actores/personas/grupos concretos por nombre.
3. **Dónde**: ubicación geográfica o contexto espacial específico. 
4. **Cuándo**: periodo temporal, fecha o momento histórico preciso.

Asignar el puntaje **RIGUROSAMENTE aplicando la siguiente escala de niveles**:

- **NIVEL 4**: Los 4 elementos están presentes Y son específicos (no generales).
- **NIVEL 3**: 3 elementos presentes o 2-3 elementos pero de forma vaga/general.
- **NIVEL 2**: Solo 1-2 elementos identificables.
- **NIVEL 1**: Ningún elemento claro o texto comienza abruptamente.

*PRECAUCIÓN*: No confundas "mencionar un tema" con "tener introducción completa".

---

## REGLA FUNDAMENTAL
Un NIVEL 4 es EXCEPCIONAL y POCO FRECUENTE. La mayoría de textos estudiantiles están entre nivel 1-3.

---

## INSTRUCCIONES CRÍTICAS

1. Evalúa la introducción en función de **los 4 elementos + la pertinencia respecto al REACTIVO**.  
2. **Si dudas entre dos niveles, elige el menor**.  
3. **NIVEL 4 es la excepción**, no la norma.  
4. **No reformules todo el texto**, solo corrige lo justo para que cumpla el nivel 4 según el reactivo.  
5. Tu corrección debe ser breve, clara y realista, sin inventar información ajena al reactivo.

---

## FORMATO DE SALIDA

Devuelve SOLO un JSON válido (sin markdown, sin comentarios adicionales):

{{
  "introduccion": {{
    "nivel": X,
    "justificacion": "Explica qué elementos faltan, están vagos o no responden al reactivo",
  }}
}}

---

## REACTIVO

{reactive}

---
## TEXTO A EVALUAR

{essay}

---

### RECORDATORIO FINAL
- Justifica cada nivel señalando DEFICIENCIAS específicas.
"""

resultados = []

for i, (texto, reactivo) in enumerate(zip(textos, reactivos), start=1):    
    prompt_introduccion_a1 = prompt_introduccion.format(essay=texto, reactive=reactivo)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_introduccion_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_introduccion.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_introduccion.json


In [7]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_con_reactivo.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()
reactivos = df["reactivo"].fillna("").tolist()


prompt_conclusion_cierre = """
Eres un evaluador y corrector EXTREMADAMENTE ESTRICTO de escritura académica. 
Tu tarea es evaluar la **CONCLUSIÓN Y CIERRE** de un texto para que cumpla con los siguientes criterios académicos exigidos y responda adecuadamente al REACTIVO planteado.

--- 

## CRITERIO DE EVALUACIÓN DE CONSLUCÓN Y CIERRE

La calidad de la conclusión y cierre se evaluará exclusivamente en función del cumplimiento de los siguientes dos aspectos:

1. **Síntesis**: reformula la idea principal del texto de forma clara, en nuevas palabras (no copiar la introducción).  
2. **Proyección**: plantea implicaciones, consecuencias, reflexiones o contextos futuros relevantes para el tema.

Asignar el puntaje **RIGUROSAMENTE aplicando la siguiente escala de niveles**:

- **NIVEL 4**: En la estructura del texto se distingue un cierre en el que se concluye o se proyecta la tesis.
- **NIVEL 3**: En la estructura del texto se distingue un cierre a modo de comentario.
- **NIVEL 2**: La estructura del texto no es clara, pero es posible inferir la presencia de una conclusión o cierre amalgamada en otras oraciones o párrafos.
- **NIVEL 1**: No se identifica una conclusión o cierre en el texto.

*PRECAUCIÓN*: una oración final simple NO equivale a una conclusión nivel 4.

---

## REGLA FUNDAMENTAL
Un NIVEL 4 es EXCEPCIONAL y POCO FRECUENTE. La mayoría de textos estudiantiles están entre nivel 1-3.

---

## INSTRUCCIONES CRÍTICAS

1. Evalúa la conclusión en función de **los 2 aspectos + la pertinencia respecto al REACTIVO**.  
2. Si dudas entre dos niveles, elige el menor.  
3. Nivel 4 es la excepción, no la norma.  
4. No inventes información nueva, pero puedes reordenar o reformular para mejorar la conclusión.  
5. La corrección sugerida debe ser breve y realista, manteniendo fidelidad al contenido y al reactivo.

---

## FORMATO DE SALIDA

Devuelve SOLO un JSON válido (sin markdown, sin comentarios adicionales):

{{
  "conclusion_cierre": {{
    "nivel": X,
    "justificacion": "Indica si falta síntesis, proyección, ambos o si conclusión no responde al reactivo"
}}

---

## REACTIVO

{reactive}

---
## TEXTO A EVALUAR

{essay}

---

### RECORDATORIO FINAL
- Justifica cada nivel señalando DEFICIENCIAS específicas.
"""

resultados = []

for i, (texto, reactivo) in enumerate(zip(textos, reactivos), start=1):    
    prompt_conclusion_cierre_a1 = prompt_conclusion_cierre.format(essay=texto, reactive=reactivo)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_conclusion_cierre_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_conclusion_cierre.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_conclusion_cierre.json


In [12]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_con_reactivo.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()
reactivos = df["reactivo"].fillna("").tolist()

prompt_progresion_textual = """
Eres un evaluador y corrector EXTREMADAMENTE ESTRICTO de escritura académica. 
Tu tarea es evaluar la **PROGRESIÓN TEXTUAL** de un texto para que cumpla con los siguientes criterios académicos exigidos y responda adecuadamente al REACTIVO planteado.

--- 

## CRITERIO DE EVALUACIÓN DE PROGRESIÓN TEXTUAL

La calidad de la progresión textual se evaluará exclusivamente en función de la presencia de los siguientes tres aspectos:

1. Cada párrafo tiene un subtema claramente distinto.
2. NO hay repetición de ideas entre párrafos.
3. Hay conectores lógicos entre párrafos.

Asignar el puntaje **RIGUROSAMENTE aplicando la siguiente escala de niveles**:

- **NIVEL 4**: El texto presenta párrafos claramente identificables y en cada uno se reconoce una información principal distinta a la del párrafo anterior.
- **NIVEL 3**: El texto presenta párrafos claramente identificables, pero en un párrafo se repite información de algún párrafo anterior.
- **NIVEL 2**: El texto presenta párrafos claramente identificables, pero en dos párrafos se repite información de algún párrafo anterior.
- **NIVEL 1**: No se presentan párrafos claramente identificables o en todos los párrafos se repite información anterior.

*PRECAUCIÓN*: Si dos párrafos dicen "básicamente lo mismo" con palabras distintas = nivel 3 máximo.

---

## REGLA FUNDAMENTAL
Un NIVEL 4 es EXCEPCIONAL y POCO FRECUENTE. La mayoría de textos estudiantiles están entre nivel 1-3.   

---

## INSTRUCCIONES CRÍTICAS

1. Evalúa la progresión textual en función de **los 3 elementos + la pertinencia respecto al REACTIVO**.  
2. **Si dudas entre dos niveles, elige el menor**.  
3. **NIVEL 4 es la excepción**, no la norma.  
4. **No reformules todo el texto**, solo corrige lo justo para que cumpla el nivel 4 según el reactivo.  
5. Tu corrección debe ser breve, clara y realista, sin inventar información ajena al reactivo.

---

## FORMATO DE SALIDA

Devuelve SOLO un JSON válido (sin markdown, sin comentarios adicionales):

{{
  "progresion_textual": {{
    "nivel": X,
    "justificacion": "Señala qué párrafos repiten información, si falta estructura o si el texto no responde al reactivo",
  }}
}}

---

## REACTIVO

{reactive}

---
## TEXTO A EVALUAR

{essay}

---

### RECORDATORIO FINAL
- Justifica cada nivel señalando DEFICIENCIAS específicas.
"""

resultados = []

for i, (texto, reactivo) in enumerate(zip(textos, reactivos), start=1):    
    prompt_progresion_textual_a1 = prompt_progresion_textual.format(essay=texto, reactive=reactivo)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_progresion_textual_a1}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "06")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "a1_progresion_textual.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\06\a1_progresion_textual.json
